### Создаем эмулятор среды ShootEnv для генерации начальных условий угла и дальности цели, а также для выдачи наград за удачные попадания.
### Награда распределена по нормальному распределению в радиусе, задаваемому в конструкторе класса

In [16]:
import numpy as np
import scipy.stats as sps

import random
from hashlib import md5
import time


class ShootEnv:
    def __init__(self, low=0, high=1000, eps=0.1, scale=1.0, radius=1.0, reward_type="norm"):
        self.eps = eps
        self.alpha = 0
        self.s = 0
        self.g = 9.8
        self.scale = scale
        self.radius = radius
        self.reward_type = reward_type
        self.low = low
        self.high = high
        

    def reset(self):
        self.alpha = np.random.uniform(low=self.eps, high=np.pi / 2 - self.eps, size=1)[0]
        self.s = np.random.uniform(low=self.low, high=self.high, size=1)[0]
        return [self.alpha, self.s]

    def step(self, action):
        s_calc = action ** 2 * np.sin(2 * self.alpha) / self.g
        if self.reward_type == "norm":
            reward = sps.norm.pdf(s_calc, loc=self.s, scale=self.scale) * 10 if np.abs(self.s - s_calc) <= self.radius else 0
        elif self.reward_type == "uniform":
            reward = 1 if np.abs(self.s - s_calc) <= self.radius else 0
        else:
            pass
        return [-1, -1], reward, True, {"real_s": s_calc}
    

#### Для решения задачи используется алгоритм Deep Deterministic Policy (DDPG), позволяющий решать задачи в непрерывных пространствах действий.

In [7]:
import torch
import torch.nn as nn

from torch.optim import Adam

torch.set_grad_enabled(True)


class DDPGAgent(nn.Module):
    def __init__(self, state_dim=2, action_dim=1, q_lr=1e-3, pi_lr=1e-3, batch_size=64, sigma=6.0, episode_n=1_000_000):
        super().__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.q = nn.Sequential(
            nn.Linear(state_dim + action_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        for module in self.q:
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight, nonlinearity="relu")

        self.pi = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim),
            nn.Sigmoid()
        )

        for module in self.pi:
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight, nonlinearity="relu")

        self.q_lr = q_lr
        self.pi_lr = pi_lr
        self.q_optimizer = Adam(self.q.parameters(), q_lr)
        self.pi_optimizer = Adam(self.pi.parameters(), pi_lr)
        self.q_scheduler = torch.optim.lr_scheduler.MultiStepLR(self.q_optimizer, milestones=[int(0.25*episode_n), int(0.6*episode_n), int(0.85*episode_n)], gamma=0.5)
        self.pi_scheduler = torch.optim.lr_scheduler.MultiStepLR(self.pi_optimizer, milestones=[int(0.3*episode_n), int(0.5*episode_n), int(0.7*episode_n), int(0.9*episode_n)], gamma=0.5)

        self.buffer = []
        self.batch_size = batch_size
        self.init_sigma = sigma
        self.sigma = sigma
        self.sigma_decay = 0.99995
        self.min_sigma = 0.1
        self.step = 0
        self.epsilon = 0
        self.alpha = 0.5
        self.init_N = 25
        self.N = self.init_N
        self.S = 20
        self.episode_n = episode_n
        # ширина диапазона возможных значений будет (0, range)
        self.range = 100

    def decay_sigma(self):
        if self.step % (self.episode_n // 4) == 0:
            self.sigma = self.init_sigma / 2
        self.sigma = max(self.sigma * self.sigma_decay, self.min_sigma)

    def decay_N(self):
        if self.step % (self.episode_n // 3) == 0:
            self.N = self.N - 5

    def decay_alpha(self):
        if self.step % (self.episode_n // 3) == 0:
            self.alpha = self.alpha * 0.75

    def disable_grads(self, net):
        for param in net.parameters():
            param.requires_grad = False

    def enable_grads(self, net):
        for param in net.parameters():
            param.requires_grad = True

    def get_action(self, obs):
        with torch.no_grad():
            obs = torch.tensor(obs, dtype=torch.float)
            # домножаем на self.range чтобы получить нужное значение
            action = self.pi(obs).numpy()[0] * self.range
            # orshteine ulunbek
            self.epsilon = self.alpha * self.epsilon + np.random.normal(loc=0, scale=self.sigma)
            noisy_action = action + self.epsilon
            return noisy_action

    def add_sample(self, state, action, reward):
        #print(state, action, reward)
        # self.buffer.append([state, [action], [reward]])
        if self.step % self.S == 0:
            self.buffer.append([state, [action], [reward]])
            
    def fit(self):
        if len(self.buffer) > self.batch_size:
            batch = random.sample(self.buffer, self.batch_size)
            states, actions, rewards = map(torch.FloatTensor, list(zip(*batch)))

            if self.step % self.N == 0:
                max_actions = self.pi(states)
                # print(max_actions.requires_grad)
                states_actions = torch.concatenate((states, max_actions), dim=1)
    
                # grads for pi model
                self.disable_grads(self.q)
                
                pi_loss = -1.0 * torch.mean(self.q(states_actions))

                pi_loss.backward()
                
                self.pi_optimizer.step()
                self.pi_optimizer.zero_grad()
    
                self.enable_grads(self.q)

            # grads for q model
            targets = rewards

            states_actions = torch.concatenate((states, actions), dim=1)
            q_loss = torch.mean((self.q(states_actions) - targets.detach())**2)
            q_loss.backward()
            self.q_optimizer.step()
            self.q_optimizer.zero_grad()
            self.step += 1
            self.decay_sigma()
            self.decay_N()
            self.decay_alpha()

            self.q_scheduler.step()
            self.pi_scheduler.step()

    def save_model(self, path=f'/home/artem/atari_games/models/DDPG_{md5(str(time.time()).encode()).hexdigest()}.pth'):
        state = {
            'q_model_dict': self.q.state_dict(),
            'pi_model_dict': self.pi.state_dict(),
            'q_optimizer_dict': self.q_optimizer.state_dict(),
            'pi_optimizer_dict': self.pi_optimizer.state_dict(),
            'epsilon': self.epsilon,
            'alpha': self.alpha,
            'sigma': self.sigma
        }
        torch.save(state, path)

    def load_model(self,path=f'/home/artem/atari_games/models/DDPG_{md5(str(time.time()).encode()).hexdigest()}'):
        if os.path.exists(path):
            state = torch.load(path)
            self.optimizer.load_state_dict(state['optimizer_dict'])
            self.q_function.load_state_dict(state['model_dict'])
            self.target_q_function.load_state_dict(state['model_dict'])
            self.epsilon = state['epsilon']
            

In [390]:
### текущие гиперпараметры  q_lr=1e-3, pi_lr=1e-4, batch_size=64, sigma=3.0 self.sigma_decay = 0.9995
# self.min_sigma = 0.8
# self.step = 0
# self.epsilon = 0
# self.alpha = 0.5
# self.N = 30
# self.S = 20

# Увеличил немного pi_lr, добавил lr_decay, увеличил время эксплорэйшна для сигмы, увеличил размеры нейросети c 32 до 64.Зафиксировал seed
# Q - функция выучена судя по нескольким примерам и довольно хорошо. Наименьший средний скор 2.5. Но агент не везде успевает выучиться.

# Пробую увеличить lr для агента. И увеличить частоту его обучения. self.N = 20. (Возможно еще попробую увеличить batch_size, но попозже)
# Увеличил пространство возможных значений для s. Не успел доучиться. Дошел до 1_000_000, но еще был потенциал к падению ошибки.

# В итоге лучшим остался скор 2.5 

# Исследования происходят волнообразно. И быстрее убывает сигма. В результате это позволяет выбираться из локальных минимумов. Увеличение батча не сработало.
# Пробую уменьшить альфу и выставить минимальную сигму поменьше.

#### Запускаю алгоритм на полтора миллиона итераций. Качество оцениваю по среднему и медианному расстоянию снаряда до цели. Также оцениваю полученнную награду.

In [17]:
torch.manual_seed(42)

env = ShootEnv(high=100, radius=1.0, reward_type="norm")

episode_n = 3_000_000

agent = DDPGAgent(episode_n=episode_n)
average = []
rewards = 0
k = 0
for i in range(episode_n):
    state = env.reset()
    
    for _ in range(10):
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        rewards += reward
        agent.add_sample(state, action, reward)
        agent.fit()
        average.append(abs(state[1] - info["real_s"]))
        if i % 1000 == 0:
            print(f'iteration: {k}, avg dist {np.round(np.mean(average), 3)}, \
            median dist {np.round(np.median(average), 3)} sum of rewards={np.round(rewards, 3)}, \
            buffer len = {len(agent.buffer)}, sanity_check last_action={round(action, 3)}')
            k += 1
            average = []
            rewards = 0
        if done:
            break
            

iteration: 0, avg dist 3.206,             median dist 3.206 sum of rewards=0,             buffer len = 1, sanity_check last_action=1.115
iteration: 1, avg dist 29.517,             median dist 23.67 sum of rewards=105.463,             buffer len = 111, sanity_check last_action=41.645
iteration: 2, avg dist 36.863,             median dist 21.152 sum of rewards=84.804,             buffer len = 161, sanity_check last_action=0.802
iteration: 3, avg dist 42.136,             median dist 19.905 sum of rewards=126.081,             buffer len = 211, sanity_check last_action=4.107
iteration: 4, avg dist 33.654,             median dist 20.144 sum of rewards=109.637,             buffer len = 261, sanity_check last_action=42.102
iteration: 5, avg dist 35.127,             median dist 18.956 sum of rewards=142.226,             buffer len = 311, sanity_check last_action=54.385
iteration: 6, avg dist 34.717,             median dist 18.154 sum of rewards=128.914,             buffer len = 361, sanity_chec

In [36]:
agent.get_action([1.1, 60])

27.001234137086435

In [37]:
27 ** 2 * np.sin(2 * 1.1) / 9.8

60.14223248821236

In [ ]:
Искусственное увеличение примеров положительного опыта приводит к тому, что происходит переобучение политики агента
1. Плохо аппроксимировалась Q-функция , увеличил количество слоев.
2. Идея - обучать сначала много Q-функцию и потом один раз делать градиентный подъем по стратегии.
3. Заменить выход стратегии на релу, чтобы не получать на выходе заведомо неверные ответы нейросети.

Пойдем по порядку.

### Итог:
### Q-функция хорошо выучила среду. В среднем обученная стратегия промахивается на +- 3 метра, однако медианное значение ниже и составляет всего 1.5 метра, что в целом приемлемо для данного этапа обучения. 

## Задача решена.